In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/team_leaders/"
data_DIR = "../data/misc/"

In [ ]:
from nba_api.stats.endpoints import leaguedashteamstats, leaguedashplayerstats


In [ ]:
stats = leaguedashteamstats.LeagueDashTeamStats(
    measure_type_detailed_defense="Base",
    per_mode_detailed="PerGame",
    date_to_nullable="01-31-2024"
)
df1 = stats.get_data_frames()[0]
df1 = df1[["TEAM_NAME","FTA"]]
stats = leaguedashteamstats.LeagueDashTeamStats(
    measure_type_detailed_defense="Base",
    per_mode_detailed="PerGame",
    date_from_nullable="02-01-2024"
)
df2 = stats.get_data_frames()[0]
df2 = df2[["TEAM_NAME","FTA"]]

In [ ]:
df3 = pd.merge(df1,df2,on="TEAM_NAME",suffixes=[" before"," after"])
df3 = df3.rename(columns={"TEAM_NAME":"Team"})
df3["Diff"] = df3["FTA after"]-df3["FTA before"]
df3 = df3.sort_values("Diff",ascending=True).reset_index(drop=True)

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i df3
df <- df3
library(tidyverse)
library(gt)
library(gtExtras)
df %>% 
  gt()%>%
  tab_header(
    title = md("**Free Throw Attempts Change after Jan 31, 2024**"),
    ) %>%
    gt_plt_bar(column = Diff, keep_column = TRUE, color = "red") %>%
    data_color(columns = c("FTA before","FTA after"), palette = "PiYG") %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 16,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2),
        table.margin.left = 1,
        table.margin.right = 1,
    ) %>% 
    tab_source_note(
    source_note = "@sradjoker | Source: nba.com/stats" ) %>% gtsave("../figs/team_leaders/ft_diff.png",size=5) 

In [ ]:
stats = leaguedashplayerstats.LeagueDashPlayerStats(
    measure_type_detailed_defense="Base",
    per_mode_detailed="PerGame",
    date_to_nullable="01-31-2024"
)
df1 = stats.get_data_frames()[0]
df1 = df1[df1["GP"]>5]
df1 = df1[["PLAYER_NAME","FTA"]]
stats = leaguedashplayerstats.LeagueDashPlayerStats(
    measure_type_detailed_defense="Base",
    per_mode_detailed="PerGame",
    date_from_nullable="02-01-2024"
)
df2 = stats.get_data_frames()[0]
df2 = df2[df2["GP"]>5]
df2 = df2[["PLAYER_NAME","FTA"]]

In [ ]:
df3 = pd.merge(df1,df2,on=["PLAYER_NAME"],suffixes=[" before"," after"])
df3 = df3.rename(columns={"PLAYER_NAME":"Player"})
df3["Diff"] = df3["FTA after"]-df3["FTA before"]
df3 = df3.sort_values("Diff",ascending=True).reset_index(drop=True)

In [ ]:
df4 = df3.nsmallest(20,"Diff")
df4 = df4.reset_index(drop=True).reset_index()
df4["index"] +=1

In [ ]:
%%R -i df4
df <- df4
library(tidyverse)
library(gt)
library(gtExtras)
df %>% 
  gt()%>%
  tab_header(
    title = md("**Free Throw Attempts Change after Jan 31, 2024**"),
    subtitle = md("Min 5 games played before and after"),
    ) %>%
    gt_plt_bar(column = Diff, keep_column = TRUE, color = "red") %>%
    data_color(columns = c("FTA before","FTA after"), palette = "PiYG") %>%
    cols_label(
      index = "#"
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 16,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2),
        table.margin.left = 1,
        table.margin.right = 1,
    ) %>% 
    tab_source_note(
    source_note = "@sradjoker | Source: nba.com/stats" ) %>% gtsave("../figs/team_leaders/ft_diff_player.png",size=5) 

In [ ]:
dfadfsd

In [ ]:
# df1 = pd.read_csv(data_DIR + "pbpstats_fouls_1.csv")
# df2 = pd.read_csv(data_DIR + "pbpstats_fouls_2.csv")
df1 = pd.read_csv(data_DIR + "pbpstats_fta_1.csv")
df2 = pd.read_csv(data_DIR + "pbpstats_fta_2.csv")

In [ ]:
df2["GamesPlayed"].sum()

In [ ]:
cols = df1.columns
for col in cols[2:]:
    df1[col] = df1[col]/df1["GamesPlayed"]
    df2[col] = df2[col]/df2["GamesPlayed"]

In [ ]:
df1["Name"] = 0
df2["Name"] = 0
df3 = pd.DataFrame(df1.mean())
df3 = df3.reset_index()
df4 = pd.DataFrame(df2.mean())
df4 = df4.reset_index()

In [ ]:
df5 = pd.merge(df3,df4,on="index")
df5 = df5.iloc[2:]
df5 = df5.reset_index(drop=True)

In [ ]:
df5.columns = ["Foul Type","Before","After"]

In [ ]:
df5["Before"] = df5["Before"].round(2)
df5["After"] = df5["After"].round(2)

In [ ]:
df5

In [ ]:
# df6 = df5.drop(8,axis=0)
# df6 = df6.drop(9,axis=0)
df6 = df5.iloc[:7]

In [ ]:
df6["Decrease"] = -(df6["After"]-df6["Before"])/df6["Before"]*100

In [ ]:
df6["Decrease"] = df6["Decrease"].round(1)

In [ ]:
%%R -i df6
df <- df6
library(tidyverse)
library(gt)
library(gtExtras)
df %>% 
  gt()%>%
  tab_header(
    title = md("**FTA Change after Jan 31, 2024**"),
    subtitle = "Change in various FTA types"
    ) %>%
    # data_color(columns = c(Decrease), palette = "PiYG",reverse=TRUE) %>%
    gt_plt_bar(column = Decrease, keep_column = TRUE, color = "red") %>%
    cols_label(
      Decrease = "Decrease [%]"
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 16,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2),
        table.margin.left = 1,
        table.margin.right = 1,
    ) %>% 
    tab_source_note(
    source_note = "@sradjoker | Source: pbpstats.com" ) %>% gtsave("../figs/team_leaders/fta_diff_league.png",size=5) 

In [ ]:
asda

In [ ]:
data_DIR1 = "../data/injuries/"

In [ ]:
year = 2023
dfinj = pd.read_parquet(data_DIR1 + f'NBA_prosptran_injuries_only_{year}.parquet')

In [ ]:
df1 = dfinj[dfinj["Date"]> "2023-10-24"]
df1 = df1[df1["Out"]]

In [ ]:
df2 = df1[df1["Date"] < "2024-02-01"]
df3 = df1[df1["Date"] > "2024-01-31"]

In [ ]:
len(df2)

In [ ]:
len(df2)/1422

In [ ]:
len(df3)

In [ ]:
len(df3)/506

# FTA Year to Year Trends

In [ ]:
from nba_api.stats.endpoints import leaguedashteamstats

In [ ]:
dfa = []
for year in tqdm(range(1996,2024)):
    season = str(year) + "-" + str(year + 1)[-2:]
    stats = leaguedashteamstats.LeagueDashTeamStats(per_mode_detailed="PerGame", season_type_all_star="Regular Season", season = season)
    df = stats.get_data_frames()[0]
    df = df[['FGM', 'FGA',
           'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
           'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS',
           'PLUS_MINUS',]]
    dfm = pd.DataFrame(df.mean()).T
    dfm["season"]=year+1
    dfa.append(dfm)
    time.sleep(1)

In [ ]:
df.columns

In [ ]:
dft = pd.concat(dfa)
dft.head()

In [ ]:
dft["FG2A"] = dft["FGA"] - dft["FG3A"]
dft["FG2M"] = dft["FGM"] - dft["FG3M"]
dft["FTA_FG2A"] = dft["FTA"]/dft["FG2A"]

In [ ]:
dft["PT2_PCT"] = (dft["FG2M"]*2)/dft["PTS"]
dft["PT3_PCT"] = (dft["FG3M"]*3)/dft["PTS"]
dft["PTF_PCT"] = (dft["FTM"]*1)/dft["PTS"]

In [ ]:
p = (
    ggplot(dft)
    + aes(x="season", y= "FTA")
    + geom_line(group=1)
    + geom_point()
    + scale_x_continuous(breaks=range(1997,2024,5))
    + theme_idv
    + labs(
            title= "FTA per Game 1996-2024",
            caption="@sradjoker | source:nba.com/stats",
        )
)
p.save(fig_DIR + "FTA_seasons.png", width=5, height=4, dpi=300)
p.draw()

In [ ]:
p = (
    ggplot(dft)
    + aes(x="season", y= "PTS")
    + geom_line(group=1)
    + geom_point()
    + scale_x_continuous(breaks=range(1997,2024,5))
    + theme_idv
    + labs(
            title= "Points Scored per Game 1996-2024",
            caption="@sradjoker | source:nba.com/stats",
        )
)
p.save(fig_DIR + "PTS_seasons.png", width=5, height=4, dpi=300)
p.draw()

In [ ]:
p = (
    ggplot(dft)
    + aes(x="season", y= "FTA_FG2A")
    + geom_line(group=1)
    + geom_point()
    + scale_x_continuous(breaks=range(1997,2024,5))
    + theme_idv
    + labs(
            title= "FTA/FG2A per Game 1996-2024",
            caption="@sradjoker | source:nba.com/stats",
            y="FTA/FG2A"
        )
)
p.save(fig_DIR + "FTA_FG2A_seasons.png", width=5, height=4, dpi=300)
p.draw()

In [ ]:
from mizani.formatters import percent_format

In [ ]:
p = (
    ggplot(dft)
    + geom_line(aes(x="season",y="PT2_PCT",color="'blue'"), group=1)
    + geom_point(aes(x="season",y="PT2_PCT",color="'blue'"))
    + geom_line(aes(x="season",y="PT3_PCT",color="'red'"), group=1)
    + geom_point(aes(x="season",y="PT3_PCT",color="'red'"))
    + geom_line(aes(x="season",y="PTF_PCT",color="'green'"), group=1)
    + geom_point(aes(x="season",y="PTF_PCT",color="'green'"))
    + scale_y_continuous(labels=percent_format())
    + scale_x_continuous(breaks=range(1997,2024,5))
    + theme_idv
    + labs(
            title= "Percentage of Points from 2s,3s and FTs 1996-2024",
            caption="@sradjoker | source:nba.com/stats",
            y="Percentage of Points"
        )
    + scale_color_identity(guide='legend',name='Shot', labels=['2s','3s','FTs'])
)
p.save(fig_DIR + "PCT_PTS_seasons.png", width=5, height=4, dpi=300)
p.draw()